In [1]:
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
XMIN = 1470
XMAX = 2840
YMIN = 50
YMAX = 2000

In [3]:
def find_centroids(th, min_size):
    if min_size < 1:
        raise ValueError('min_size must be at least 1')
    
    _, cont, _ = cv2.findContours(th, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    M = map(cv2.moments, cont)
    
    cxs = []
    cys = []

    for i, m in enumerate(M):
        if m['m00'] < min_size:
            continue

        cx = int(m['m10']/m['m00'])
        cy = int(m['m01']/m['m00'])

        cxs.append(cx)
        cys.append(cy)
      
    return cxs, cys

In [4]:
def get_masked_window(grayed, cx, cy, size):
    ymin = cy - size // 2 if cy - size // 2 > 0 else 0
    xmin = cx - size // 2 if cx - size // 2 > 0 else 0
    windowed = grayed[cy-size//2:cy+size//2, cx-size//2:cx+size//2]
    _, bw = cv2.threshold(windowed, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    _, cont, _ = cv2.findContours(bw, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    if 0 < len(cont) < 3:
        return windowed * (bw == 0) / 256.
    else:
        return None

## TF model

In [5]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# CNN
SIZE = 28

tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=[None, SIZE, SIZE])
y_ = tf.placeholder(tf.float32, [None, 3])

with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, SIZE, SIZE, 1])
    
with tf.name_scope('conv'):
    W_conv = weight_variable([3, 3, 1, 8])
    b_conv = bias_variable([8])
    h_conv = tf.nn.relu(tf.nn.conv2d(x_image, W_conv, strides=[1, 1, 1, 1], padding='SAME'))
    
with tf.name_scope('pool'):
    h_pool = tf.nn.max_pool(h_conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
with tf.name_scope('fc1'):
    W_fc1 = weight_variable([SIZE//2*SIZE//2*8, 100])
    b_fc1 = bias_variable([100])
        
    h_pool_flat = tf.reshape(h_pool, [-1, SIZE//2*SIZE//2*8])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat, W_fc1) + b_fc1)
    
with tf.name_scope('fc2'):
    W_fc2 = weight_variable([100, 3])
    b_fc2 = bias_variable([3])

    y = tf.matmul(h_fc1, W_fc2) + b_fc2
    
saver = tf.train.Saver()

sess = tf.InteractiveSession()
saver.restore(sess, '../data/tf_save/trained_model.ckpt')

INFO:tensorflow:Restoring parameters from ../data/tf_save/trained_model.ckpt


In [6]:
MIN_CONTOUR_SIZE = 45
WINDOW_SIZE = SIZE

INPUT_VIDEO_FILE = '../data/videos/TandemRun.mp4'
OUTPUT_VIDEO_FILE = '../data/videos/out.mp4'

video = cv2.VideoCapture(INPUT_VIDEO_FILE)
if not video.isOpened():
    raise IOError('video "{}" not open'.format(INPUT_VIDEO_FILE))

out_video = cv2.VideoWriter(OUTPUT_VIDEO_FILE, cv2.VideoWriter_fourcc('X', '2', '6', '4'), 30, (3840, 2160))

start_time = time.time()

frame_count = 0
while frame_count // 30 < 40:
    ret, frame = video.read()
    
    if not ret:
        print('End of video.')
        break

    frame_count += 1
    if frame_count % 30 == 0:
        print('Processed {}s of video. Time Elapsed {}s'.format(frame_count//30, time.time()-start_time))
        start_time = time.time()
      
    cropped = frame[YMIN:YMAX, XMIN:XMAX]
    grayed = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
    th = cv2.adaptiveThreshold(grayed, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)
    cxs, cys = find_centroids(th, MIN_CONTOUR_SIZE)
    
    masks = []
    positions = []
    
    for cx, cy in zip(cxs, cys):
        masked = get_masked_window(grayed, cx, cy, WINDOW_SIZE)
        if masked is not None and masked.size == WINDOW_SIZE * WINDOW_SIZE:
            masks.append(masked)
            positions.append((cx, cy))
    # Feed masked windows to trained model for prediction.
    predictions = sess.run(tf.argmax(y, 1), feed_dict={x: masks})
    for prediction, (cx, cy) in zip(predictions, positions):
        if prediction == 1:
            cv2.rectangle(frame, (cx+XMIN-15, cy+YMIN-15), (cx+XMIN+15, cy+YMIN+15), (0, 255, 0), 3)
        # elif prediction == 2:
        #     cv2.rectangle(frame, (cx+XMIN-15, cy+YMIN-15), (cx+XMIN+15, cy+YMIN+15), (255, 0, 0), 3)
       
    out_video.write(frame)

out_video.release()

Processed 1s of video. Time Elapsed 7.9103827476501465s
Processed 2s of video. Time Elapsed 7.340809106826782s
Processed 3s of video. Time Elapsed 7.58121919631958s
Processed 4s of video. Time Elapsed 7.551974773406982s
Processed 5s of video. Time Elapsed 7.470412015914917s
Processed 6s of video. Time Elapsed 7.639024019241333s
Processed 7s of video. Time Elapsed 7.646846055984497s
Processed 8s of video. Time Elapsed 7.532934904098511s
Processed 9s of video. Time Elapsed 7.4754533767700195s
Processed 10s of video. Time Elapsed 7.528733968734741s
Processed 11s of video. Time Elapsed 7.786754846572876s
Processed 12s of video. Time Elapsed 7.638563871383667s
Processed 13s of video. Time Elapsed 7.59133505821228s
Processed 14s of video. Time Elapsed 7.818426132202148s
Processed 15s of video. Time Elapsed 7.649585008621216s
Processed 16s of video. Time Elapsed 7.800814151763916s
Processed 17s of video. Time Elapsed 7.70005202293396s
Processed 18s of video. Time Elapsed 7.732223033905029s
Pr

In [11]:
sess.close()